In [1]:
## load the libraries 
%matplotlib inline
import sys
import warnings
import os
import glob
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split

from keras.layers import *
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.models import Model, Sequential
from keras.metrics import *
from keras.optimizers import Adam, RMSprop
from scipy.stats import norm
from keras.preprocessing import image
from keras import datasets

from keras import backend as K
import random
import matplotlib.pyplot as plt
from skimage import exposure
plt.gray()

os.environ["CUDA_VISIBLE_DEVICES"]="0" #model will be trained on GPU 0

Using TensorFlow backend.


<Figure size 432x288 with 0 Axes>

In [2]:
def load_image(path):
    x_size = 64
    y_size = 64
    image_list = np.zeros((len(path), x_size, y_size,1))
    for i, fig in enumerate(path):
        img = image.load_img(fig, color_mode='grayscale', target_size=(x_size, y_size))
        img = image.img_to_array(img).astype('float32')
        img = img / 255.0
        '''
        max_i = img.max()
        min_i = img.min()

        img = (img - min_i) / (max_i - min_i)
        '''
        
        image_list[i] = img#exposure.equalize_adapthist(img[:,:,0])
        
    return image_list

In [3]:
"""
train_img = glob.glob("/home/webvalley/Documents/plaque-image-preprocessing/Data/Patches/Plaque/Aug/cal/*.png")
train_img += glob.glob("/home/webvalley/Documents/plaque-image-preprocessing/Data/Patches/Plaque/Aug/fibrous/*.png")
train_img += random.sample(glob.glob("/home/webvalley/Documents/plaque-image-preprocessing/Data/Patches/NoPlaque/*.png"), 3790)
"""
train_img = glob.glob("train/*.png")

test_img = glob.glob("test/*.png")


random.shuffle(train_img)
random.shuffle(test_img)

x_train = load_image(train_img)
x_test = load_image(test_img) 

In [4]:
for i, img in enumerate(x_train):
    max_i = img.max()
    min_i = img.min()
    if max_i == min_i:
        img = np.zeros((64,64,1))
    else:
        img = (img - min_i) / (max_i - min_i)
        img = exposure.equalize_adapthist(img[:,:,0])
    
    x_train[i] = img.reshape(64,64,1)

KeyboardInterrupt: 

In [ ]:
plt.imshow(x_train[0].reshape(64,64))
plt.show()
plt.imshow(img.reshape(64,64))
plt.show()

In [ ]:
"""
imgs = glob.glob("/home/webvalley/Documents/plaque-image-preprocessing/AutoEncoder/test/*.png")
print(imgs[len(imgs)-1])


for _ in range(200):
    r = random.randint(0, len(imgs)-1)
    name = imgs[r].split("/")[-1]
    while not "noplaque" in name:
        r = random.randint(0, len(imgs))
        name = imgs[r].split("/")[-1]
    
    print('mv "' + imgs[r] + '" /home/webvalley/Documents/plaque-image-preprocessing/AutoEncoder/train')

    os.system('mv "' + imgs[r] + '" /home/webvalley/Documents/plaque-image-preprocessing/AutoEncoder/train')


for im in glob.glob("/home/webvalley/Documents/plaque-image-preprocessing/Data/Patches/Plaque/Aug/cal/*.png"):
    #print(im)
    os.system(f"cp {im} train")
"""

In [ ]:
y_test = []

for img in test_img:
    if "Calcific" in img:
        y_test.append(0)
    elif "Fibrous" in img:
        y_test.append(1)
    elif "noplaque" in img:
        y_test.append(2)

In [ ]:
batch_size = 128
epochs = 20
inChannel = 1
x, y = 64, 64
input_img = Input(shape = (x, y, inChannel))
print(input_img.shape)

In [ ]:
img_rows, img_cols, img_chns = x, y, 1
# number of convolutional filters to use
NB_FILTERS = 16
# convolution kernel size
KERNEL_SIZE = (3, 3)


LATENT_DIM = 128
INTERMEDIATE_DIM = 256
NB_EPOCHS = 100

conv_1 = Conv2D(1, kernel_size=(2, 2), padding='same', activation='relu')(input_img)
conv_2 = Conv2D(NB_FILTERS, kernel_size=(2, 2),
                       padding='same', activation='relu',
                       strides=(2, 2))(conv_1)
conv_3 = Conv2D(NB_FILTERS, KERNEL_SIZE,
                       padding='same', activation='relu',
                       strides=(1, 1))(conv_2)
conv_4 = Conv2D(NB_FILTERS, KERNEL_SIZE,
                       padding='same', activation='relu',
                       strides=(1, 1))(conv_3)
flat = Reshape((-1,))(conv_4)
hidden = Dense(INTERMEDIATE_DIM, activation='relu')(flat)

z = Dense(LATENT_DIM)(hidden)

decoder_hid = Dense(INTERMEDIATE_DIM, activation='relu')(z)
decoder_upsample = Dense(NB_FILTERS * img_rows//2 * img_rows//2, activation='relu')(decoder_hid)

decoder_reshape = Reshape((img_rows//2, img_rows//2, NB_FILTERS))(decoder_upsample)
decoder_deconv_1 = Conv2DTranspose(NB_FILTERS, KERNEL_SIZE,
                                   padding='same',
                                   strides=(1, 1),
                                   activation='relu')(decoder_reshape)
decoder_deconv_2 = Conv2DTranspose(NB_FILTERS, KERNEL_SIZE,
                                   padding='same',
                                   strides=(1, 1),
                                   activation='relu')(decoder_deconv_1)

decoder_deconv_3_upsamp = Conv2DTranspose(NB_FILTERS, kernel_size=(2, 2),
                                          padding='valid',
                                          strides=(2, 2),
                                          activation='relu')(decoder_deconv_2)
decoder_mean_squash = Conv2D(img_chns, kernel_size=(1, 1),
                                    padding='valid',
                                    activation='sigmoid')(decoder_deconv_3_upsamp)


In [ ]:
"""FUNZIONANTE
# Encoder
encoded = Conv2D(2,(3,3),
           activation='relu',
           padding='same')(input_img)
encoded = Conv2D(2,(3,3),
           activation='relu',
           padding='same')(encoded)
encoded = MaxPooling2D((2,2))(encoded) 
encoded = Conv2D(4,(3,3),
           activation='relu',
           padding='same')(encoded)
encoded = Conv2D(4,(3,3),
           activation='relu',
           padding='same')(encoded)
encoded = MaxPooling2D((2,2))(encoded) 
encoded = Flatten()(encoded)
encoded = Dense(64, activation='relu')(encoded)
encoded = Dense(4)(encoded)

# Decoder
decoded = Dense(64, activation='relu')(encoded)
decoded = Dense(1024, activation='relu')(decoded)
decoded = Reshape((7,7,4))(decoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(4, (3, 3),
           activation='relu',
           padding='same')(decoded)
decoded = Conv2D(2, (3, 3),
           activation='relu',
           padding='same')(decoded)
decoded = UpSampling2D((2, 2))(decoded) 
decoded = Conv2D(2, (3, 3),
           activation='relu',
           padding='same')(decoded)
decoded = Conv2D(1, (3, 3),
           padding='same')(decoded)
"""

In [ ]:
"""
# encoding architecture
encoded = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
print(encoded.shape)
encoded = MaxPool2D( (2, 2), padding='same')(encoded)
encoded = Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
encoded = MaxPool2D( (2, 2), padding='same')(encoded)
encoded = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
encoded = MaxPool2D((2, 2), padding='same')(encoded)
encoded = Flatten()(encoded)
encoded = Dense(128)(encoded)
encoded = Dense(2)(encoded)
"""

In [ ]:
"""
# decoding architecture
decoded = Dense(256)(encoded)
decoded = Reshape((4, 4, 16))(decoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(16, (3, 3), activation='relu', padding='same')(decoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(32, (3, 3), activation='relu', padding='same')(decoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(64, (3, 3), activation='relu', padding='same')(decoded)
decoded = UpSampling2D((2, 2))(decoded)
decoded = Conv2D(1, (3, 3), padding='same')(decoded)
"""

In [ ]:

smooth = 1

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)


In [ ]:
# compile the model

model = Model(input_img, decoder_mean_squash)
model.compile(optimizer=Adam(lr=0.0001), loss='mse')#dice_coef, metrics=[dice_coef_loss])
model.summary()

In [ ]:
#early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=5, mode='auto')
history = model.fit(x_train, x_train, epochs=100, batch_size=batch_size, validation_data=(x_test, x_test))#, callbacks=[early_stopping])

In [ ]:
from keras.models import load_model

model.save('autoencoder_model.h5') 

In [ ]:
# compile the model

# adam = optimizers.Adam(lr=0.01)

encoder = Model(input_img, z)
encoder.compile(optimizer=Adam(lr=0.0001), loss='mse')#dice_coef, metrics=[dice_coef_loss])

In [ ]:
img = x_train[0]

In [ ]:
img.shape

In [ ]:

encoded_imgs = encoder.predict(x_test)
decoded_imgs = model.predict(x_test)

In [ ]:
encoded_imgs.shape
#print(encoded_imgs[0].shape)

In [ ]:
encoded_imgs[0].shape

In [ ]:
encoded_imgs[0]

In [ ]:
label_conv = {
    0: 'Calcific',
    1: 'Fibrous',
    2: 'NoPlaque'
}

In [ ]:

for i in range(11):
    fig, ((ax1, ax2, ax3)) = plt.subplots(1, 3)
    
    fig.suptitle(f"[{i}] Label: " + label_conv[y_test[i]], fontsize=16)
    
    ax1.imshow(x_test[i].reshape((64, 64)))
    ax1.set_title("Original")
    
    ax1.get_xaxis().set_visible(False)
    ax1.get_yaxis().set_visible(False)
    
    ax2.imshow(encoded_imgs[i].reshape((16, 8)))
    ax2.set_title("Encoded")
    
    ax2.get_xaxis().set_visible(False)
    ax2.get_yaxis().set_visible(False)
    
    ax3.imshow(decoded_imgs[i].reshape((64, 64)))
    ax3.set_title("Decoded")
    
    ax3.get_xaxis().set_visible(False)
    ax3.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cos_sim = cosine_similarity(encoded_imgs[2].reshape(1,-1), encoded_imgs[1].reshape(1,-1))

In [ ]:
cos_sim

In [ ]:
np.sum(np.subtract(encoded_imgs[0], encoded_imgs[5]))

In [ ]:
#encoded_imgs[np.array(y_test) == 1]
plt.title("SCATTER")
plt.scatter(encoded_imgs[np.array(y_test) == 0][:, 0], encoded_imgs[np.array(y_test) == 0][:, 1], label='Calcific', alpha=0.5)
plt.scatter(encoded_imgs[np.array(y_test) == 1][:, 0], encoded_imgs[np.array(y_test) == 1][:, 1], label='Fibrous', alpha=0.5)
plt.scatter(encoded_imgs[np.array(y_test) == 2][:, 0], encoded_imgs[np.array(y_test) == 2][:, 1], label='No Plaque', alpha=0.5)
plt.legend();

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
proj = pca.fit_transform(encoded_imgs)


In [ ]:
len(proj)

In [ ]:
len(pca.components_)

In [ ]:
pca.explained_variance_

In [ ]:
plt.title("PCA")
plt.scatter(proj[np.array(y_test) == 0][:, 0], proj[np.array(y_test) == 0][:, 1], label='Calcific', alpha=0.5)
plt.scatter(proj[np.array(y_test) == 1][:, 0], proj[np.array(y_test) == 1][:, 1], label='Fibrous', alpha=0.5)
plt.scatter(proj[np.array(y_test) == 2][:, 0], proj[np.array(y_test) == 2][:, 1], label='No Plaque', alpha=0.5)
plt.legend();

In [ ]:
import umap


standard_embedding = umap.UMAP(random_state=42).fit_transform(encoded_imgs)
"""
plt.scatter(standard_embedding[:, 0], standard_embedding[:, 1], c=y_test, cmap='Spectral', alpha=0.5)
plt.colorbar();
"""

In [ ]:
standard_embedding

In [ ]:
plt.title("UMAP")
plt.scatter(standard_embedding[np.array(y_test) == 0][:, 0], standard_embedding[np.array(y_test) == 0][:, 1], alpha=0.5)
plt.scatter(standard_embedding[np.array(y_test) == 1][:, 0], standard_embedding[np.array(y_test) == 1][:, 1], alpha=0.5)
plt.scatter(standard_embedding[np.array(y_test) == 2][:, 0], standard_embedding[np.array(y_test) == 2][:, 1], alpha=0.5);

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=3,perplexity=40).fit_transform(encoded_imgs)

In [ ]:
plt.title("T-SNE")
plt.scatter(tsne[np.array(y_test) == 0][:, 0], tsne[np.array(y_test) == 0][:, 1], alpha=0.5)
plt.scatter(tsne[np.array(y_test) == 1][:, 0], tsne[np.array(y_test) == 1][:, 1], alpha=0.5)
plt.scatter(tsne[np.array(y_test) == 2][:, 0], tsne[np.array(y_test) == 2][:, 1], alpha=0.5);